In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#col_list=['학수번호-분반', '강좌명', '대상학년', '정원', '교수명','학점', 
 #         '강의시간/강의실', '이수구분', '언어구분', '특이사항','대학']
#Class Code	Class	Year	Quota	
#Instructor	Credit	Class Hour/Room	Type	Language	Notice	Syllabus
col_list=['class_code','class_name','class_year','quota','instructor','credit',
          'class_hour_room','class_type','class_lan','notice','campus']
result = pd.DataFrame(columns=(col_list))
#result.loc[len(result)]=list_

In [3]:
options = webdriver.ChromeOptions ()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
chrome_driver=webdriver.Chrome('./datas/chromedriver',chrome_options=options)
chrome_driver.implicitly_wait(3)

## 국제캠 모든 과목 크롤링

In [4]:
for i in range(1,80):
    print(i)
    chrome_driver.get('http://gaya1.khu.ac.kr/servlets/timetable?attribute=frame_kor&fake=Wed%20Sep%2019%202018%2013:58:55%20GMT+0900%20(%C7%D1%B1%B9%20%C7%A5%C1%D8%BD%C3)')
    #초기하면
    chrome_driver.switch_to_frame(chrome_driver.find_element_by_xpath('/html/frameset/frame[1]'))
    # 프레임변환 위쪽 변환 창으로
    chrome_driver.find_element_by_xpath('//*[@id="article"]/div[1]/div/table/tbody/tr[1]/td[3]/ul/li/p/select/option[1]').click()
    #option1 국제캠임 option2 서울캠
    chrome_driver.find_element_by_xpath('//*[@id="p_major"]/option[{0}]'.format(i)).click()
    #option 1~90 서울캠 1~79 국제캠
    chrome_driver.find_element_by_xpath('//*[@id="article"]/div[1]/div/table/tbody/tr[2]/td[6]/div/span/button').click()
    #조회버튼
    chrome_driver.switch_to_default_content()
    # 초기 화면으로
    chrome_driver.switch_to_frame(chrome_driver.find_element_by_xpath('/html/frameset/frame[2]'))
    # 아랫쪽 리스트 창으로 전환하기
    html = chrome_driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    table_body=soup.find('tbody')
    rows = table_body.find_all('tr')
    if rows[0].text.strip()!="데이터가 존재하지 않습니다." :
        for row in rows:
            cols=row.find_all('td')
            cols=[x.text.strip() for x in cols]
            cols.pop()
            cols.append("국제")
            result.loc[len(result)]=cols

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79


## 서울캠 모든 과목 크롤링

In [5]:
for i in range(1,91):
    print(i)
    chrome_driver.get('http://gaya1.khu.ac.kr/servlets/timetable?attribute=frame_kor&fake=Wed%20Sep%2019%202018%2013:58:55%20GMT+0900%20(%C7%D1%B1%B9%20%C7%A5%C1%D8%BD%C3)')
    #초기하면
    chrome_driver.switch_to_frame(chrome_driver.find_element_by_xpath('/html/frameset/frame[1]'))
    # 프레임변환 위쪽 변환 창으로
    chrome_driver.find_element_by_xpath('//*[@id="article"]/div[1]/div/table/tbody/tr[1]/td[3]/ul/li/p/select/option[2]').click()
    #option1 국제캠임 option2 서울캠
    chrome_driver.find_element_by_xpath('//*[@id="p_major"]/option[{0}]'.format(i)).click()
    #option 1~90 서울캠 1~79 국제캠
    chrome_driver.find_element_by_xpath('//*[@id="article"]/div[1]/div/table/tbody/tr[2]/td[6]/div/span/button').click()
    #조회버튼
    chrome_driver.switch_to_default_content()
    # 초기 화면으로
    chrome_driver.switch_to_frame(chrome_driver.find_element_by_xpath('/html/frameset/frame[2]'))
    # 아랫쪽 리스트 창으로 전환하기
    html = chrome_driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    table_body=soup.find('tbody')
    rows = table_body.find_all('tr')
    if rows[0].text.strip()!="데이터가 존재하지 않습니다." :
        for row in rows:
            cols=row.find_all('td')
            cols=[x.text.strip() for x in cols]
            cols.pop()
            cols.append("서울")
            result.loc[len(result)]=cols

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90


In [7]:
result.shape

(5020, 11)

In [8]:
result=result.drop_duplicates(keep='first')
result.shape

(4961, 11)

In [9]:
def merge_row(group_series):
    if (group_series==group_series.iloc[0]).all():
        return group_series.iloc[0]
    else:
        group_series=group_series.astype(str)
        #print(group_series)
        return group_series.str.cat(sep=',')

In [10]:
result=result.groupby('class_code').agg(merge_row)
result.shape

(4320, 10)

In [11]:
result.reset_index(level=0, inplace=True)

In [12]:
result

,class_code,class_name,class_year,quota,instructor,credit,class_hour_room,class_type,class_lan,notice,campus
0,ACCT1001-01,회계원리,1,80,심규영,3.0,월 09:00-10:15 (오422)수 09:00-10:15 (오422),11,,,서울
1,ACCT1001-02,회계원리,1,85,심규영,3.0,월 10:30-11:45 (오422)수 10:30-11:45 (오422),11,,,서울
2,ACCT1001-03,회계원리,1,40,강병민,3.0,월 15:00-16:15 (오307)수 15:00-16:15 (오307),11,,,서울
3,ACCT1001-04,회계원리,1,60,김권중,3.0,월 15:00-16:15 (오211)수 15:00-16:15 (오211),11,,,서울
4,ACCT1001-05,회계원리,1,40,강병민,3.0,월 16:30-17:45 (오307)수 16:30-17:45 (오307),11,,,서울
5,ACCT1001-06,회계원리,1,60,김권중,3.0,월 16:30-17:45 (오211)수 16:30-17:45 (오211),11,,,서울
6,ACCT1001-07,회계원리,1,60,허귀진,3.0,화 09:00-11:45 (외국어대306),11,,국제캠퍼스 개설강좌,서울
7,ACCT1001-08,회계원리,1,60,김성은,3.0,화 10:30-11:45 (오211)목 10:30-11:45 (오211),11,,,서울
8,ACCT1001-09,회계원리,1,60,김성은,3.0,화 12:00-13:15 (오211)목 12:00-13:15 (오211),11,,,서울
9,ACCT1001-10,회계원리,1,60,김성은,3.0,화 13:30-14:45 (오211)목 13:30-14:45 (오211),11,,,서울


In [73]:
result.to_csv('./datas/khu_courses_eng.csv',index=False,encoding='utf8')

In [14]:
#result.to_csv('./datas/khu_courses_eng_noheader.csv',index=False,header=False,encoding='utf8')

In [15]:
#result.head(10).to_csv('./datas/khu_courses_test.csv',index=False,encoding='utf8')

In [16]:
#result.head(10).to_csv('./datas/khu_courses_test_noheader.csv',index=False,header=False,encoding='utf8')

In [17]:
result.head(10)

,class_code,class_name,class_year,quota,instructor,credit,class_hour_room,class_type,class_lan,notice,campus
0,ACCT1001-01,회계원리,1,80,심규영,3.0,월 09:00-10:15 (오422)수 09:00-10:15 (오422),11,,,서울
1,ACCT1001-02,회계원리,1,85,심규영,3.0,월 10:30-11:45 (오422)수 10:30-11:45 (오422),11,,,서울
2,ACCT1001-03,회계원리,1,40,강병민,3.0,월 15:00-16:15 (오307)수 15:00-16:15 (오307),11,,,서울
3,ACCT1001-04,회계원리,1,60,김권중,3.0,월 15:00-16:15 (오211)수 15:00-16:15 (오211),11,,,서울
4,ACCT1001-05,회계원리,1,40,강병민,3.0,월 16:30-17:45 (오307)수 16:30-17:45 (오307),11,,,서울
5,ACCT1001-06,회계원리,1,60,김권중,3.0,월 16:30-17:45 (오211)수 16:30-17:45 (오211),11,,,서울
6,ACCT1001-07,회계원리,1,60,허귀진,3.0,화 09:00-11:45 (외국어대306),11,,국제캠퍼스 개설강좌,서울
7,ACCT1001-08,회계원리,1,60,김성은,3.0,화 10:30-11:45 (오211)목 10:30-11:45 (오211),11,,,서울
8,ACCT1001-09,회계원리,1,60,김성은,3.0,화 12:00-13:15 (오211)목 12:00-13:15 (오211),11,,,서울
9,ACCT1001-10,회계원리,1,60,김성은,3.0,화 13:30-14:45 (오211)목 13:30-14:45 (오211),11,,,서울


In [18]:
import pymysql
from sqlalchemy import create_engine

In [ ]:
pw_=input("pw : ")

In [68]:
pymysql.install_as_MySQLdb()

In [69]:
engine = create_engine("mysql+pymysql://fingp:"+pw_+
                       "@175.195.89.200:3306/fingpDB_test?charset=utf8", encoding='utf-8')

In [70]:
conn = engine.connect()

In [71]:
result.to_sql(name='course_tb', con=engine, if_exists='append', index=False)

In [72]:
conn.close()